# VAE version 001

In [2]:
import numpy as np
import torch
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.functional as F

In [3]:
data_input_size = (46, 1000)

Define network

In [26]:
class VGGStyleEncoder(nn.Module):
    def __init__(self, embedding_size):
        super(VGGStyleEncoder, self).__init__()
        self.features = nn.Sequential(
            # conv1
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, padding='same'),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # conv2
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding='same'),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # conv3
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding='same'),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding='same'),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # conv4
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding='same'),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, padding='same'),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.flatten = nn.Flatten()
        
        # Calculate the flattened size after the convolutional layers
        self.fc_mu = nn.Linear(512 * 2 * 62, embedding_size)
        self.fc_logvar = nn.Linear(512 * 2 * 62, embedding_size)
        
    def forward(self, x):
        x = self.features(x)
        x = self.flatten(x)
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

class Decoder(nn.Module):
    def __init__(self, embedding_size):
        super(Decoder, self).__init__()
        self.fc = nn.Linear(embedding_size, 512 * 2 * 62)
        
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, kernel_size=4, stride=2, padding='same'),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding='same'),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding='same'),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 1, kernel_size=4, stride=2, padding='same'),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.fc(x)
        x = x.view(-1, 512, 2, 62)
        x = self.decoder(x)
        return x

class VAE(nn.Module):
    def __init__(self, embedding_size):
        super(VAE, self).__init__()
        self.encoder = VGGStyleEncoder(embedding_size)
        self.decoder = Decoder(embedding_size)
        
    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decoder(z)
        return recon_x, mu, logvar

# Example usage
embedding_size = 128  # Define the embedding size
vae = VAE(embedding_size)

# Example input: single channel, 1x46x1000
input_data_2d = torch.randn(10, 1, 46, 1000)  # Adjust input shape to (batch_size, channels, height, width)
recon_x, mu, logvar = vae(input_data_2d)  # Adjust for 2D input
print(recon_x.shape)
# Display the input image
input_image = input_data_2d[0, 0].detach().numpy()
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(input_image, cmap='viridis', aspect='auto')
plt.colorbar()
plt.title('Input Image')
plt.xlabel('Width')
plt.ylabel('Height')

# Display the reconstructed output image
output_image = recon_x[0, 0].detach().numpy()
plt.subplot(1, 2, 2)
plt.imshow(output_image, cmap='viridis', aspect='auto')
plt.colorbar()
plt.title('Reconstructed Output Image')
plt.xlabel('Width')
plt.ylabel('Height')

plt.show()

TypeError: conv_transpose2d(): argument 'padding' (position 5) must be tuple of ints, but found element of type str at pos 0

In [21]:
# Define the training loop
num_epochs = 10  # Number of epochs to train
learning_rate = 0.001  # Learning rate

# Define the optimizer and loss function
optimizer = torch.optim.Adam(vae.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

# Training loop
for epoch in range(num_epochs):
    vae.train()
    optimizer.zero_grad()
    
    # Forward pass
    recon_x, mu, logvar = vae(input_data_2d)
    
    # Compute the reconstruction loss
    recon_loss = criterion(recon_x, input_data_2d)
    
    # Compute the KL divergence
    kl_div = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    # Total loss
    loss = recon_loss + kl_div
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    # Print the loss for each epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Display the final reconstructed output image
vae.eval()
with torch.no_grad():
    recon_x, _, _ = vae(input_data_2d)
    output_image = recon_x[0, 0].detach().numpy()
    plt.imshow(output_image, cmap='viridis', aspect='auto')
    plt.colorbar()
    plt.title('Final Reconstructed Output Image')
    plt.xlabel('Width')
    plt.ylabel('Height')
    plt.show()

RuntimeError: The size of tensor a (992) must match the size of tensor b (1000) at non-singleton dimension 3